In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('task4_insurance.csv')
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [3]:
print('Кол-во строк: {}\nКол-во столбцов: {}\nКол-во строк без пропусков:\n{}'.format(len(df),len(df.columns),df.count()))

Кол-во строк: 1338
Кол-во столбцов: 7
Кол-во строк без пропусков:
age         1338
sex         1338
bmi         1338
children    1338
smoker      1338
region      1338
charges     1338
dtype: int64


In [4]:
df_uniq = pd.concat([df.nunique().to_frame().rename({0:'unique'},axis=1),df.count().to_frame().rename({0:'all'},axis=1)], axis=1)
df_uniq

,unique,all
age,47,1338
sex,2,1338
bmi,548,1338
children,6,1338
smoker,2,1338
region,4,1338
charges,1337,1338


In [5]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [6]:
df.groupby(by = ['sex','smoker'])['charges'].agg(['mean','median']).unstack()

mean                    median            
smoker           no           yes          no         yes
sex                                                      
female  8762.297300  30678.996276  7639.41745  28950.4692
male    8087.204731  33042.005975  6985.50695  36085.2190

In [7]:
df.groupby(by = ['sex','children'])['charges'].agg(['mean']).unstack()

mean                                            \
children             0             1             2             3   
sex                                                                
female    11905.714276  12161.360414  13941.317326  13865.605066   
male      12832.696736  13273.522458  16187.095325  16789.167419   

                                     
children             4            5  
sex                                  
female    13937.674562  9854.006419  
male      13782.284829  7931.658310

In [8]:
df.groupby(by = ['sex','region'])['charges'].agg(['mean']).unstack()

mean                                          
region     northeast     northwest     southeast     southwest
sex                                                           
female  12953.203151  12479.870397  13499.669243  11274.411264
male    13854.005374  12354.119575  15879.617173  13412.883576

In [9]:
df.groupby(by = ['smoker','region'])['charges'].agg(['mean']).unstack()

mean                                          
region     northeast     northwest     southeast     southwest
smoker                                                        
no       9165.531672   8556.463715   8032.216309   8019.284513
yes     29673.536473  30192.003182  34844.996824  32269.063494

Вывод: курить - дорого, особенно, если живешь на юго востоке

быть мужчиной, если куришь и детей < 4 - тоже дорого

---
Подготовка данных

---

In [10]:
import sklearn
from sklearn.model_selection import train_test_split

In [11]:
X = df.drop('charges', axis = 1)
y = df['charges']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3,random_state= 1)

In [13]:
X_train_number = X_train.select_dtypes(include=[np.number])
X_train_cat = X_train.select_dtypes(exclude=[np.number])
display(X_train_number.head(10),X_train_cat.head(10))

,age,bmi,children
744,50,26.410,0
363,21,26.400,1
10,25,26.220,0
970,50,28.160,3
634,51,39.700,1
554,25,41.325,0
314,27,31.400,0
419,63,26.980,0
525,18,33.880,0
1041,18,23.085,0


,sex,smoker,region
744,male,no,northwest
363,female,no,southwest
10,male,no,northeast
970,female,no,southeast
634,male,no,southwest
554,female,no,northeast
314,female,yes,southwest
419,female,yes,northwest
525,female,no,southeast
1041,male,no,northeast


In [14]:
X_train_cat['sex'] = X_train_cat['sex'].apply(lambda a: 1 if a == "female" else 0)
X_train_cat['smoker'] = X_train_cat['smoker'].apply(lambda x: 1 if x == 'yes' else 0)

In [15]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
encoded = encoder.fit_transform(X_train_cat[["region"]])
X_train_cat_full = pd.concat([X_train_cat.drop(columns=['region']).reset_index(drop=True), 
           pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names_out())], axis=1)

In [16]:
X_train_cat_full

,sex,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,0,0,0.0,1.0,0.0,0.0
1,1,0,0.0,0.0,0.0,1.0
2,0,0,1.0,0.0,0.0,0.0
3,1,0,0.0,0.0,1.0,0.0
4,0,0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
931,0,0,0.0,0.0,0.0,1.0
932,1,0,1.0,0.0,0.0,0.0
933,1,1,1.0,0.0,0.0,0.0
934,1,1,0.0,0.0,1.0,0.0


In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_number_norm = scaler.fit_transform(X_train_number)

In [18]:
X_train_full = pd.concat([pd.DataFrame(X_train_number_norm, columns=scaler.get_feature_names_out()), X_train_cat_full], axis=1)

In [19]:
X_train_full

,age,bmi,children,sex,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,0.797152,-0.702114,-0.904002,0,0,0.0,1.0,0.0,0.0
1,-1.271085,-0.703758,-0.085679,1,0,0.0,0.0,0.0,1.0
2,-0.985811,-0.733340,-0.904002,0,0,1.0,0.0,0.0,0.0
3,0.797152,-0.414510,1.550967,1,0,0.0,0.0,1.0,0.0
4,0.868471,1.482037,-0.085679,0,0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
931,1.510338,-0.292894,-0.904002,0,0,0.0,0.0,0.0,1.0
932,-0.914493,-0.218117,0.732644,1,0,1.0,0.0,0.0,0.0
933,0.868471,0.703039,0.732644,1,1,1.0,0.0,0.0,0.0
934,0.083967,-1.390722,0.732644,1,1,0.0,0.0,1.0,0.0


In [20]:
means = scaler.mean_
stds = np.sqrt(scaler.var_)

In [21]:
X_numbers_check =  X_train_full[['age','bmi','children']].head(10)
for i in range(len(X_numbers_check.columns)):
    item = X_numbers_check.columns[i]
    X_numbers_check[item] = X_numbers_check[item].apply(lambda a: a*stds[i]+means[i])


In [22]:
display(X_train_number.head(10),X_numbers_check)

,age,bmi,children
744,50,26.410,0
363,21,26.400,1
10,25,26.220,0
970,50,28.160,3
634,51,39.700,1
554,25,41.325,0
314,27,31.400,0
419,63,26.980,0
525,18,33.880,0
1041,18,23.085,0


,age,bmi,children
0,50.0,26.410,0.0
1,21.0,26.400,1.0
2,25.0,26.220,0.0
3,50.0,28.160,3.0
4,51.0,39.700,1.0
5,25.0,41.325,0.0
6,27.0,31.400,0.0
7,63.0,26.980,0.0
8,18.0,33.880,0.0
9,18.0,23.085,0.0


In [23]:
X_test_number = X_test.select_dtypes(include=[np.number])
X_test_cat = X_test.select_dtypes(exclude=[np.number])
display(X_test_number.head(10),X_test_cat.head(10))

,age,bmi,children
559,19,35.530,0
1087,57,31.540,0
1020,51,37.000,0
460,49,36.630,3
802,21,22.300,1
298,31,34.390,3
481,49,37.510,2
616,56,28.595,0
763,27,26.030,0
750,37,26.400,0


,sex,smoker,region
559,male,no,northwest
1087,male,no,northwest
1020,male,no,southwest
460,female,no,southeast
802,male,no,southwest
298,male,yes,northwest
481,male,no,southeast
616,female,no,northeast
763,male,no,northeast
750,female,yes,southeast


In [24]:
X_test_cat['sex'] = X_test_cat['sex'].apply(lambda a: 1 if a == "female" else 0)
X_test_cat['smoker'] = X_test_cat['smoker'].apply(lambda x: 1 if x == 'yes' else 0)

In [25]:
encoded_test = encoder.transform(X_test_cat[["region"]])
X_test_cat_full = pd.concat([X_test_cat.drop(columns=['region']).reset_index(drop=True), 
           pd.DataFrame(encoded_test.toarray(), columns=encoder.get_feature_names_out())], axis=1)

In [26]:
X_test_cat_full

,sex,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,0,0,0.0,1.0,0.0,0.0
1,0,0,0.0,1.0,0.0,0.0
2,0,0,0.0,0.0,0.0,1.0
3,1,0,0.0,0.0,1.0,0.0
4,0,0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
397,0,0,1.0,0.0,0.0,0.0
398,1,0,0.0,0.0,0.0,1.0
399,1,0,1.0,0.0,0.0,0.0
400,0,1,0.0,0.0,1.0,0.0


In [27]:
X_test_number_norm = scaler.transform(X_test_number)

In [28]:
X_test_full = pd.concat([pd.DataFrame(X_test_number_norm, columns=scaler.get_feature_names_out()), X_test_cat_full], axis=1)

In [29]:
X_test_full

,age,bmi,children,sex,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,-1.413722,0.796716,-0.904002,0,0,0.0,1.0,0.0,0.0
1,1.296382,0.140978,-0.904002,0,0,0.0,1.0,0.0,0.0
2,0.868471,1.038304,-0.904002,0,0,0.0,0.0,0.0,1.0
3,0.725834,0.977496,1.550967,1,0,0.0,0.0,1.0,0.0
4,-1.271085,-1.377574,-0.085679,0,0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
397,1.296382,1.686647,-0.904002,0,0,1.0,0.0,0.0,0.0
398,-1.342404,0.430226,-0.904002,1,0,0.0,0.0,0.0,1.0
399,-1.342404,-0.311794,-0.904002,1,0,1.0,0.0,0.0,0.0
400,-0.914493,-0.595290,-0.904002,0,1,0.0,0.0,1.0,0.0


In [30]:
X_numbers_check_2 =  X_test_full[['age','bmi','children']].head(10)
for i in range(len(X_numbers_check_2.columns)):
    item = X_numbers_check_2.columns[i]
    X_numbers_check_2[item] = X_numbers_check_2[item].apply(lambda a: a*stds[i]+means[i])
display(X_test_number.head(10),X_numbers_check_2)

,age,bmi,children
559,19,35.530,0
1087,57,31.540,0
1020,51,37.000,0
460,49,36.630,3
802,21,22.300,1
298,31,34.390,3
481,49,37.510,2
616,56,28.595,0
763,27,26.030,0
750,37,26.400,0


,age,bmi,children
0,19.0,35.530,0.0
1,57.0,31.540,0.0
2,51.0,37.000,0.0
3,49.0,36.630,3.0
4,21.0,22.300,1.0
5,31.0,34.390,3.0
6,49.0,37.510,2.0
7,56.0,28.595,0.0
8,27.0,26.030,0.0
9,37.0,26.400,0.0


---
Обучение

---

In [31]:
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [32]:
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=1)
param_grid = {
    'loss' : ['squared_error'],
    'penalty': ['elasticnet'],  
    'alpha': [0.00001, 0.0001,0.001,0.01,0.05,0.1],
    'l1_ratio': [0, 0.1, 0.3, 0.5, 0.7, 0.9, 1],  
}

In [48]:
param_grid = {
   'max_depth' : [8,7,6, 5, 4, 3]
}

In [33]:
sgd_regressor = SGDRegressor()

In [46]:
import catboost as ct

In [49]:
cat_boost = ct.CatBoostRegressor(random_state=1, allow_writing_files=False, thread_count=-1, verbose=False)

In [51]:
grid_search_mse = GridSearchCV(cat_boost, param_grid, cv=5, scoring="neg_mean_squared_error")
grid_search_mse.fit(X_train_full,y_train)

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x000002052943AD30>,
             param_grid={'max_depth': [8, 7, 6, 5, 4, 3]},
             scoring='neg_mean_squared_error')

In [52]:
pd.DataFrame(grid_search_mse.cv_results_).sort_values(by='rank_test_score',axis = 0).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,0.957489,0.119385,0.003229,0.006212,3,{'max_depth': 3},-2.709269e+07,-1.695999e+07,-2.268591e+07,-2.454447e+07,-2.076164e+07,-2.240894e+07,3.434514e+06,1
4,1.279794,0.059520,0.005060,0.006085,4,{'max_depth': 4},-2.740058e+07,-1.823865e+07,-2.279161e+07,-2.446713e+07,-2.125416e+07,-2.283043e+07,3.070361e+06,2
3,1.692739,0.152742,0.006132,0.005814,5,{'max_depth': 5},-2.804672e+07,-1.917634e+07,-2.356476e+07,-2.536436e+07,-2.163350e+07,-2.355714e+07,3.043494e+06,3
2,2.520186,0.331752,0.004458,0.003859,6,{'max_depth': 6},-2.901372e+07,-2.006874e+07,-2.301830e+07,-2.673366e+07,-2.099318e+07,-2.396552e+07,3.407749e+06,4
1,3.177417,0.180169,0.003612,0.006830,7,{'max_depth': 7},-2.939731e+07,-2.072540e+07,-2.419927e+07,-2.697191e+07,-2.180863e+07,-2.462050e+07,3.212566e+06,5


In [53]:
best_params_mse = grid_search_mse.best_params_
best_params_mse

{'max_depth': 3}

In [58]:
best_model_mse = grid_search_mse.best_estimator_

In [38]:
grid_search_mae = GridSearchCV(sgd_regressor, param_grid, cv=cv_strategy, scoring="neg_mean_absolute_error")
grid_search_mae.fit(X_train_full,y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             estimator=SGDRegressor(),
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.05, 0.1],
                         'l1_ratio': [0, 0.1, 0.3, 0.5, 0.7, 0.9, 1],
                         'loss': ['squared_error'], 'penalty': ['elasticnet']},
             scoring='neg_mean_absolute_error')

In [39]:
best_params_mae = grid_search_mae.best_params_
best_model_mae = grid_search_mae.best_estimator_
best_params_mae

{'alpha': 0.0001,
 'l1_ratio': 0.9,
 'loss': 'squared_error',
 'penalty': 'elasticnet'}

In [40]:
grid_search_r2 = GridSearchCV(sgd_regressor, param_grid, cv=cv_strategy, scoring="r2")
grid_search_r2.fit(X_train_full,y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             estimator=SGDRegressor(),
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.05, 0.1],
                         'l1_ratio': [0, 0.1, 0.3, 0.5, 0.7, 0.9, 1],
                         'loss': ['squared_error'], 'penalty': ['elasticnet']},
             scoring='r2')

In [41]:
best_params_r2 = grid_search_r2.best_params_
best_model_r2 = grid_search_r2.best_estimator_
best_params_r2

{'alpha': 0.001,
 'l1_ratio': 0.5,
 'loss': 'squared_error',
 'penalty': 'elasticnet'}

In [42]:
baseline_reg = SGDRegressor(penalty = None)

In [43]:
baseline_reg.fit(X_train_full,y_train)

SGDRegressor(penalty=None)

In [59]:
preds_mse = best_model_mse.predict(X_test_full)
preds_mae = best_model_mae.predict(X_test_full)
preds_r2 = best_model_r2.predict(X_test_full)
preds_base =baseline_reg.predict(X_test_full)

In [60]:
print('----------------metrics = mse------------------')
print('Тестовая средння абсолютная ошибка: {}'.format(mean_absolute_error(y_test,preds_mse)))
print('Тестовая среднеквадратичная ошибка: {}'.format(mean_squared_error(y_test,preds_mse)))
print('Тестовый r2: {}'.format(r2_score(y_test,preds_mse)))
print('Тестовое отношение mae к среднему: {}%'.format(round((mean_absolute_error(y_test,preds_mse)/y_test.mean())*100,2)))
print('Тестовое средняя цена: {}'.format(y_test.mean()))
print('----------------metrics = mae------------------')
print('Тестовая средння абсолютная ошибка: {}'.format(mean_absolute_error(y_test,preds_mae)))
print('Тестовая среднеквадратичная ошибка: {}'.format(mean_squared_error(y_test,preds_mae)))
print('Тестовый r2: {}'.format(r2_score(y_test,preds_mae)))
print('Тестовое отношение mae к среднему: {}%'.format(round((mean_absolute_error(y_test,preds_mae)/y_test.mean())*100,2)))
print('Тестовое средняя цена: {}'.format(y_test.mean()))
print('----------------metrics = r2------------------')
print('Тестовая средння абсолютная ошибка: {}'.format(mean_absolute_error(y_test,preds_r2)))
print('Тестовая среднеквадратичная ошибка: {}'.format(mean_squared_error(y_test,preds_r2)))
print('Тестовый r2: {}'.format(r2_score(y_test,preds_r2)))
print('Тестовое отношение mae к среднему: {}%'.format(round((mean_absolute_error(y_test,preds_r2)/y_test.mean())*100,2)))
print('Тестовое средняя цена: {}'.format(y_test.mean()))
print('----------------базавая модель------------------')
print('Тестовая средння абсолютная ошибка: {}'.format(mean_absolute_error(y_test,preds_base)))
print('Тестовая среднеквадратичная ошибка: {}'.format(mean_squared_error(y_test,preds_base)))
print('Тестовый r2: {}'.format(r2_score(y_test,preds_base)))
print('Тестовое отношение mae к среднему: {}%'.format(round((mean_absolute_error(y_test,preds_base)/y_test.mean())*100,2)))
print('Тестовое средняя цена: {}'.format(y_test.mean()))

----------------metrics = mse------------------
Тестовая средння абсолютная ошибка: 2520.7389933836034
Тестовая среднеквадратичная ошибка: 20071636.40777363
Тестовый r2: 0.8583678547173061
Тестовое отношение mae к среднему: 19.02%
Тестовое средняя цена: 13255.80881669154
----------------metrics = mae------------------
Тестовая средння абсолютная ошибка: 4142.502298942089
Тестовая среднеквадратичная ошибка: 36719467.03451366
Тестовый r2: 0.7408952222888441
Тестовое отношение mae к среднему: 31.25%
Тестовое средняя цена: 13255.80881669154
----------------metrics = r2------------------
Тестовая средння абсолютная ошибка: 4139.833400481368
Тестовая среднеквадратичная ошибка: 36717946.88839214
Тестовый r2: 0.7409059489457026
Тестовое отношение mae к среднему: 31.23%
Тестовое средняя цена: 13255.80881669154
----------------базавая модель------------------
Тестовая средння абсолютная ошибка: 4142.149890280825
Тестовая среднеквадратичная ошибка: 36724689.16465968
Тестовый r2: 0.740858373200882

После нескольких инициализаций пришёл к выводу, что все модели дают примерно одинаковый результат. На этих данных регуляризация почти не нужна